In [ ]:
%matplotlib inline
import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import nept
from core import Rat, combine_rats
from load_data import assign_label
from plotting import plot_behavior, plot_duration

In [ ]:
data_filepath = 'E:/code/emi_biconditional/cache/data/spring2017'

In [ ]:
sessions = ['!2017-05-04', '!2017-05-05']

In [ ]:
# rats = ['R141', 'R142', 'R143', 'R144', 'R145', 'R146', 'R147', 'R148']
rats = ['R141']
groups = [1, 2, 2, 1, 2, 1, 1, 2]
males = ['R141', 'R143', 'R145', 'R147']
females = ['R142', 'R144', 'R146', 'R148']
group1 = ['R141', 'R144', 'R146', 'R147']
group2 = ['R142', 'R143', 'R145', 'R148']

cue_duration = 10
iti_buffer = 2

data = dict()
for rat in rats:
    data[rat] = Rat(rat, group1, group2)

for session in sessions:
    rats_data = nept.load_medpc(os.path.join(data_filepath, session), assign_label)

    for rat in rats:
        iti_starts = []
        iti_stops = []
        for trial in ['trial1', 'trial2', 'trial3', 'trial4']:
            iti_starts.extend(rats_data[rat][trial].starts - cue_duration - iti_buffer)
            iti_stops.extend(rats_data[rat][trial].starts - iti_buffer)
        rats_data[rat]['iti'] = nept.Epoch(np.vstack([iti_starts, iti_stops]))

    for rat, group in zip(rats, groups):
        data[rat].add_session(**rats_data[rat], group=group)

n_sessions = len(data[rats[0]].sessions)

df = combine_rats(data, rats, n_sessions)

In [ ]:
len(df)

In [ ]:
long = df[df['session'] == 1]
short = df[df['session'] == 2]

In [ ]:
len(np.unique(long['trial']))

In [ ]:
np.sum(long['session'] == 1)

In [ ]:
np.sum(short['session'] == 2)

In [ ]:
len(np.unique(short['trial']))

In [ ]:
long

In [ ]:
short

In [ ]:
output_filepath = 'E:/code/emi_biconditional/cache/'

for rat in rats:
    filename = rat + '_outcome_behavior.png'
    filepath = os.path.join(output_filepath, filename)
    plot_behavior(df, [rat], filepath=None, by_outcome=True)

In [ ]:
single_session = df[df['session'] == 1].copy()

if int(len(single_session) / (3 * 4)) == 32: # 3 light, sound, iti. 4 measures.
    rat = single_session['rat'][0]
    trials = []
    for trial in range(len(single_session)):
        trials.append('%s, %d' % (rat, int(trial/4)+96))
    single_session['trial'] = trials

this_df = pd.concat([df, single_session], ignore_index=True)

In [ ]:
np.unique(df['session'])

In [ ]:
this_df[380:400]

In [ ]:
len(df[df['session'] == 2])

In [ ]:
384 / 4

In [ ]:
384 / 32

In [ ]:
3 * 4

In [ ]:
df